This is my model that gave me first place on the NCAAW Spread Competition. There isn't really much to see here, except for a twist at the end. It's simple mapping from my probabilities in the main contest to spreads based on the historical data included.

Since this is almost all there is to it, let me also briefly describe the model that gave me the probabilities and 39th place (and my second silver) in the other competition.

My first and so far only March Madness competitions were in 2018. I really could not care less about almost all sports, including basketball. So definitely no domain knowledge entered this model. Like most competitors, I concentrated on my NCAAM model in 2018, when the women's competition ran for the first time. My men's model is based on Elo scores and Kenpom pre-tournament ratings. It hasn't been doing so well, but I still have some hope to edge my way into the low bronze ranks there.

With little time to spare for the women's competition in 2018 and no Kenpom information available, the model is a simple Elo rating converted to win probabilities in the usual way. I tuned the home advantage, auto-correlation factor, and reversion between seasons by a rough grid search, separetely for men and women.

My Elo computation runs over the regular season only, tournament games and post-season games are simply never considered, all for the sake of simplicity and development speed. Also all teams start out with an Elo rating of 1500, no corrections is made for expansion or contraction of divisions or conferences.

I missed out on the 2019 version of this competition, mostly because I was under the mistaken impression that they wouldn't count towards medals and points, as was the case in the years before. Being too busy I couldn't bring myself to invest time in kaggle. I got ready to participate again in 2020, but we all know what happened then.

Expecting another year of COVID-19 I didn't even bother to check whether March Madness would happen until I saw some tweet indicating that it would. Then I checked kaggle and with four days to go for the men's tournament I decided to dust off my models from 2018 and simply run them again with some fine-tuning but no new development. That gave me enough time to submit both to the men's and women's main competion with about two days to spare before the deadline each. I thought then that I was done. Only late on Saturday evening, I noticed the spread competitions. The deadline for the men's spread competition was already over but I thought a quick model for the women's competition should be easy enough to attempt on Sunday with a few hours to spare.

I got leave from my family to do this for ~1.5 hours on Sunday noon about four hours before the deadline. I quickly hacked together what you see here and with 10 minutes to spare I submitted the winning solution. The time pressure probably explains the bug at the end.

This is my first kaggle competition win, and most definitely a very unexpected one, given the story above!

Although I don't manage to be as active on here as I would like, I'm very grateful for the community from whom I have learned a lot!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

from pathlib import Path

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Start by loading past Tournament results and my NCAAW probabilites without finals flipped and no clipping applied. Compute the win margins with the right signs (lower TeamID first).

In [ ]:
data_dir = Path("/kaggle/input/ncaaw-march-mania-2021-spread/WDataFiles_Stage2_Spread/")
tourney_results = pd.read_csv(data_dir / "WNCAATourneyDetailedResults.csv")
tourney_results = tourney_results.assign(Margin=lambda x: 
                                         np.sign(x.LTeamID - x.WTeamID) * (x.WScore - x.LScore))
prob_df = pd.read_csv("/kaggle/input/2021-ncaaw-solution-39th-place/submit_stage1_elo_tuned.csv")

Check that the margins and especially their signs look good.

In [ ]:
tourney_results[["Season", "WTeamID", "LTeamID", "WScore", "LScore", "Margin"]].head(20)

Split the ID from the stage 1 main competition submission file to have Season, T1_TeamID, and T2_TeamID. We will later merge on these keys.

In [ ]:
def get_year_t1_t2(ID):
    """Return a tuple with ints `year`, `team1` and `team2`."""
    return (int(x) for x in ID.split('_'))


for ii, row in prob_df.iterrows():
    year, t1, t2 = get_year_t1_t2(row.ID)
    prob_df.loc[ii, "Season"] = year
    prob_df.loc[ii, 'T1_TeamID'] = t1
    prob_df.loc[ii, 'T2_TeamID'] = t2
prob_df["Season"] = prob_df["Season"].astype(int)
prob_df["T1_TeamID"] = prob_df["T1_TeamID"].astype(int)
prob_df["T2_TeamID"] = prob_df["T2_TeamID"].astype(int)
prob_df.head()

Also add T1/2_TeamID to the tournament results.

In [ ]:
for ii, row in tourney_results.iterrows():
    t1_id, t2_id = row.WTeamID, row.LTeamID
    if t1_id > t2_id:
        t2_id, t1_id = t1_id, t2_id
    tourney_results.loc[ii, "T1_TeamID"] = t1_id
    tourney_results.loc[ii, "T2_TeamID"] = t2_id
    tourney_results.loc[ii, "Margin"] = row.Margin
tourney_results["T1_TeamID"] = tourney_results["T1_TeamID"].astype(int)
tourney_results["T2_TeamID"] = tourney_results["T2_TeamID"].astype(int)
tourney_results.head(10)

And then merge the stage1 submission with the tournament results.

In [ ]:
merged = prob_df.merge(tourney_results, on=["T1_TeamID", "T2_TeamID"])
merged.plot("Pred", "Margin", kind="scatter");

Looks noisy but the trend is right. At this point I decided not to find a functional form a fit a spline but thought about taking a windowed average. How large a window? Well that led me to using KNN and tuning k via GridSearchCV.

In [ ]:
knn = KNeighborsRegressor()
param_grid = dict(n_neighbors=np.arange(1, 50))
clf = GridSearchCV(knn, param_grid)
clf.fit(merged.Pred.to_numpy().reshape(-1, 1), merged.Margin)
clf.best_index_

And now use the best model with k=41 to predict. This is what I submitted and gave me first place.

In [ ]:
stage2_df = pd.read_csv("/kaggle/input/2021-ncaaw-solution-39th-place/elo_2021-04-05.csv")
pred_margin = clf.best_estimator_.predict(stage2_df.Pred.to_numpy().reshape(-1, 1))
stage2_df["Pred"] = pred_margin
stage2_df[["ID", "Pred"]].to_csv("/kaggle/working/elo_calibrated_margin_submitted.csv", index=False)

Let's check this notebook against my final score.

In [ ]:
stage2_result = pd.read_csv("/kaggle/input/2021-ncaaw-solution-39th-place/2021NCAAWTourneyMarginResults.csv")
pred_results_combined = stage2_result.merge(stage2_df, on="ID", how="left")
print(f"Final score: {mean_squared_error(pred_results_combined.Margin, pred_results_combined.Pred, squared=False)}")

Notebook score and leaderboard match.

But wait? Have you noticed anything above? I didn't for a few days.

There are way to many tournament games in the merged frame. I forgot to merge on Season, too! I guess I am very lucky that Women's College Basketball is very stable over many years and the noise this mistake introduced is not catastrophic.

So, what would have happened had I noticed my error and corrected it before the submission deadline? Let's see and do the merge correctly:

In [ ]:
merged = prob_df.merge(tourney_results, on=["T1_TeamID", "T2_TeamID", "Season"])
merged.plot("Pred", "Margin", kind="scatter");


This already looks much better and less noisy.

In [ ]:
knn = KNeighborsRegressor()
param_grid = dict(n_neighbors=np.arange(1, 50))
clf = GridSearchCV(knn, param_grid)
clf.fit(merged.Pred.to_numpy().reshape(-1, 1), merged.Margin)
clf.best_index_

In [ ]:
stage2_df = pd.read_csv("/kaggle/input/2021-ncaaw-solution-39th-place/elo_2021-04-05.csv")
pred_margin = clf.best_estimator_.predict(stage2_df.Pred.to_numpy().reshape(-1, 1))
stage2_df["Pred"] = pred_margin
stage2_df[["ID", "Pred"]].to_csv("/kaggle/working/elo_calibrated_margin_fix_season_merge.csv", index=False)

In [ ]:
pred_results_combined = stage2_result.merge(stage2_df, on="ID", how="left")
print(f"Fixed score: {mean_squared_error(pred_results_combined.Margin, pred_results_combined.Pred, squared=False)}")

Oops, I could have won more decisively. Well, first place is first place.

And let's have a quick look at the model:

In [ ]:
x = np.linspace(0, 1, 101)
pred = clf.best_estimator_.predict(x.reshape(-1, 1))
plt.plot(x, pred)
plt.xlabel("Win Probability")
plt.ylabel("Margin")
plt.hlines(0, 0, 1, linestyle="dotted", colors="k")
plt.vlines(0.5, -30, 45, linestyle="dotted", colors="k");

I noticed on submission that my model is asymmetric in its predicted margin. I had no time to do anything about it. I checked later and any attempt to symmetrize it I attempted after the deadline would have cost me first place.

There is a lot of noise in these competitions and to some degree I got lucky with the asymmetry. Although there is just a chance asymmetry in the data as well. Alphabetically lower teams in the NCAAW are just a little stronger than alphabetically higher teams. Forcing your model to ignore this is then probably not a good idea.

A few words on the competition metric, where I also got lucky. The blowout wins are particularly hard to predict and easily dominate the RMSE. The highest margin my model would predict is 43.24 and two games finished with a margin of 43 and one with a margin of 42. Maybe, if you don't want you metric dominated by the mostly unevenly matched first and second round games, something like MAD or fractional RMSE would be better metrics for this competition.

Thanks for reading!